In [1]:
import argparse
import sys
import time
import random
import traceback
from collections import deque
from pprint import pprint
import wandb
import numpy as np
from decimal import Decimal

In [2]:
import torch.optim as optim
from mlagents_envs.environment import UnityEnvironment, ActionTuple
from mlagents_envs.side_channel.environment_parameters_channel import EnvironmentParametersChannel

In [3]:
from gymnasium import spaces 
from stable_baselines3.common.buffers import ReplayBuffer

from training_utils import *
from testing_utils import *

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [4]:
import argparse
import sys

def parse_args(default_config_path="./config/test.yaml"):
    """
    Parse arguments from CLI or notebook.
    - In notebook: usa il default se non passato
    - In CLI: permette override dei parametri nel config
    """
    # --- Gestione notebook: evita crash su ipykernel args ---
    argv = sys.argv[1:]
    # Se siamo in notebook o non è passato il config_path, inseriamo il default
    if len(argv) == 0 or "--f=" in " ".join(argv):
        argv = [default_config_path]

    # --- Pre-parser per leggere il config_path ---
    pre_parser = argparse.ArgumentParser(add_help=False)
    pre_parser.add_argument(
        "config_path",
        type=str,
        nargs="?",
        default=default_config_path,
        help="Main config file path"
    )
    initial_args, remaining_argv = pre_parser.parse_known_args(argv)
    CONFIG_PATH = initial_args.config_path
    print(f"Config path: {CONFIG_PATH}")

    # --- Legge parametri dal file di config ---
    file_config_dict = parse_config_file(CONFIG_PATH)

    # --- Parser principale ---
    parser = argparse.ArgumentParser(description="Training Script")
    parser.add_argument(
        "config_path",
        type=str,
        nargs="?",
        default=CONFIG_PATH,
        help="Main config file path"
    )

    # Aggiunge parametri dal config file, con tipi corretti
    for key, value in file_config_dict.items():
        if isinstance(value, bool):
            parser.add_argument(f"--{key}", type=str2bool, default=value)
        elif value is None:
            parser.add_argument(f"--{key}", type=str, default=value)
        else:
            parser.add_argument(f"--{key}", type=type(value), default=value)

    # --- Parse finale con remaining_argv per ignorare args extra Jupyter ---
    args, unknown = parser.parse_known_args(remaining_argv)
    if unknown:
        print("Ignored unknown args:", unknown)
    return args


# Testing Function

In [5]:
def test(env, 
         env_info,
         param_channel,
         
         args, 
         agent_config,
         obstacles_config,
         
         actor,
         
         BEHAVIOUR_NAME,
         STATE_SIZE,
         RAYCAST_SIZE,
         DEVICE
        ):
    
    print('Applying Unity settings from config...')
    apply_unity_settings(param_channel, agent_config, 'ag_')
    apply_unity_settings(param_channel, obstacles_config, 'obs_')

    print('Resetting environment...')
    env.reset() 
    env_info.clear_queue()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    
    # Precompute ray angles for CBF checks
    '''angoli_radianti_precalcolati = generate_angles_rad(
        10,
        90
    )'''

    start_time = time.time()
    unity_end_time = -1
    unity_start_time = -1
    testing_stats = {
        "python_time": RunningMean(),
        "unity_time": RunningMean(),
    }
    
    current_episode = 1
    cumulative_obs = {}          # per-agent memory (obs, action, uncertainty info)
    running_episodes = {}        # active episodes data
    terminated_episodes = []    # finished episodes
    
    episodic_stats = {}
    dataset = []                 # collected dataset
        
    while current_episode <= args.total_episodes:
        
        try:  
            # --- ENVIRONMENT STEP ---
            unity_start_time = time.time()
            if unity_end_time > 0:
                testing_stats['python_time'].update(unity_start_time - unity_end_time)
            
            env.step()
            
            
            unity_end_time = time.time()
            testing_stats['unity_time'].update(unity_end_time - unity_start_time)
            
            obs = collect_data_after_step_id(env, BEHAVIOUR_NAME, STATE_SIZE)
            
            for id in obs:
                agent_obs = obs[id]

                # Handle terminated agents
                if agent_obs[3] == 1:
                    if id in cumulative_obs:
                        # Remove agent from active lists and finalize episode
                        del cumulative_obs[id]
                        terminated_episodes.append((agent_obs[4], running_episodes[id])) # tuple (internal_id, episode data)
                        del running_episodes[id]
                        
                    else:
                        # Agent killed very early
                        terminated_episodes.append((agent_obs[4], []))
                        assert id not in running_episodes and id not in cumulative_obs
                        
                else:
                    actual_obs = agent_obs[0]
                        
                    # Initialize new agent entry
                    if id not in cumulative_obs:
                        cumulative_obs[id] = [
                            args.decision_frame_period, # steps until next decision
                            None,   # last obs
                            None,   # last action taken
                        ]
                        
                    # Time to decide an action
                    if cumulative_obs[id][0] >= args.decision_frame_period:
                        cumulative_obs[id][0] = 0
                        
                        # Update ray observations with frame stacking
                        if cumulative_obs[id][1] is None:
                            cumulative_obs[id][1] = actual_obs
                            corrected_obs = actual_obs
                        else:
                            corrected_obs = cumulative_obs[id][1][RAYCAST_SIZE + STATE_SIZE:] 
                            corrected_obs = np.concatenate([corrected_obs, actual_obs[-RAYCAST_SIZE - STATE_SIZE:]])

                        # Policy action from actor
                        action, _, _ = actor.get_action(torch.Tensor([corrected_obs]).to(DEVICE))
                        action = action[0].detach().cpu().numpy()
                        
                        
                        '''# Uncertainty filter (optional)
                        if CONFIG_DICT['uncertainty_filter']['enabled']: 
                            uncertanty_estimate = filter_methods[CONFIG_DICT['uncertainty_filter']['method']](
                                corrected_obs, 
                                action
                            )
                            cumulative_obs[id][4] = uncertanty_estimate
                            cumulative_obs[id][5] = uncertanty_estimate > CONFIG_DICT['uncertainty_filter']['threshold']'''
                        
                        # Update agent memory
                        cumulative_obs[id][1] = corrected_obs
                        cumulative_obs[id][2] = action
                        
                        # Start new episode if not already tracked
                        if id not in running_episodes: running_episodes[id] = []
                        running_episodes[id].append({
                            'obs': corrected_obs,
                            # 'u_e': cumulative_obs[id][4],
                            # 'uf_activation': cumulative_obs[id][5],
                            'action': action,
                            'inner_steps': []
                        })

                    # Use last predicted action by default
                    policy_action = cumulative_obs[id][2] 
                    
                    # Control Barrier Function (CBF) correction
                    '''
                    cbf_action = np.zeros(2)
                    if CONFIG_DICT['cbf']['enabled']:
                        if CONFIG_DICT['uncertainty_filter']['application'] != 'dynamic':
                            cbf_action = CBF_from_obs(
                                actual_ray_obs[-1], policy_action, env_info,
                                CONFIG_DICT['cbf']['d_safe'],
                                CONFIG_DICT['cbf']['alpha'],
                                CONFIG_DICT['cbf']['d_safe_mul'],
                                angoli_radianti_precalcolati
                            )

                        else:
                            cbf_action = CBF_from_obs(
                                actual_ray_obs[-1], policy_action, env_info,
                                CONFIG_DICT['cbf']['d_safe'] * min(cumulative_obs[id][4]/CONFIG_DICT['uncertainty_filter']['threshold'], 1),
                                CONFIG_DICT['cbf']['alpha'],
                                CONFIG_DICT['cbf']['d_safe_mul'],
                                angoli_radianti_precalcolati
                            )
                            
                        # Ensure minimum forward velocity
                        if policy_action[0] > CONFIG_DICT['cbf']['min_forward']:
                            cbf_action[0] = max(CONFIG_DICT['cbf']['min_forward'], cbf_action[0])
                        else:
                            cbf_action[0] = max(policy_action[0], cbf_action[0])'''
                                
                    # Check if CBF activated
                    # cbf_activation = CONFIG_DICT['cbf']['enabled'] and np.linalg.norm(cbf_action - policy_action) > 0.0001
                    # running_episodes[id][-1]['inner_steps'].append([np.linalg.norm(cbf_action - policy_action), cbf_activation])
                    
                    # Final action selection (UF + CBF logic)
                    final_action = policy_action
                    '''if CONFIG_DICT['uncertainty_filter']['application'] == 'interpolation':
                        interpolation_coeff = min(cumulative_obs[id][4]/CONFIG_DICT['uncertainty_filter']['threshold'], 1)
                        final_action = cbf_action * interpolation_coeff + ( 1- interpolation_coeff) * policy_action
                    else:  
                        if cumulative_obs[id][5] and cbf_activation:
                            final_action = cbf_action'''
                    
                    # Debug visualization (optional)
                    if args.send_debug_action:
                        env_info.send_agent_action_debug(
                            agent_obs[4],
                            
                            final_action[0], 
                            final_action[1],
                            
                            policy_action[0],
                            policy_action[1], 
                            
                            False, 
                            0.0, 
                            0.0,
                            
                            False,
                            0.0,
                            0.0
                        ) 
                                                            
                    # Apply final action to environment
                    a = ActionTuple(continuous=np.array([final_action]))
                    env.set_action_for_agent(
                        BEHAVIOUR_NAME, id, a
                    )
                    
                    # Increment frame counter
                    cumulative_obs[id][0] += 1
        except Exception as e:
            print('Execution Ended ?!')
            traceback.print_exc() 
            
        new_stop_msgs = []
        for msg in env_info.stop_msg_queue:
            
            t_episode = None
            t_episode_index = -1
            for external_id, (internal_id, episode_data) in enumerate(terminated_episodes):
                if internal_id == msg['id']:
                    t_episode = episode_data
                    t_episode_index = external_id
                    break
            if t_episode is None:
                new_stop_msgs.append(msg)
                continue
            # Process terminated episode
            

            if t_episode == []:
                print(current_episode, '- agent killed too early, step', msg['length'])
                print(msg)
            else:
                update_stats_from_message_rm(episodic_stats, None, None, msg)        
                if current_episode % args.metrics_log_interval == 0:
                    print_update(current_episode, args.total_episodes, start_time, episodic_stats)
                
                current_episode += 1
                
                # Save data if required
                # saving all obseravtion + action and episode stats
                if args.accumulate_data:
                    dataset.append(([
                        list(element['obs']) + list(element['action'])
                        for element in t_episode
                    ], msg))
            
            print(current_episode, msg)
            del terminated_episodes[int(t_episode_index)]
        env_info.stop_msg_queue = new_stop_msgs
              
        # Safety check: queue should not grow indefinitely
        if len(env_info.stop_msg_queue) > 10:
            print('ERRORE')
            raise AssertionError('Unexpected queue growth')
        
    episodic_stats["unity_time"] = testing_stats["unity_time"]
    episodic_stats["python_time"] = testing_stats["python_time"]
    return episodic_stats, dataset


# Start Testing Code

In [6]:
args = parse_args()
train_config = parse_config_file(args.train_config_path)
agent_config = parse_config_file(args.agent_config_path)
obstacles_config = parse_config_file(train_config["obstacles_config_path"])
other_config = parse_config_file(train_config["other_config_path"])

# args.seed = random.randint(0, 2**16)
# args.name = generate_funny_name()

print('Testing with the following parameters:')
pprint(vars(args))

print('train_config:')
pprint(train_config)

print('agent_config:')
pprint(agent_config)

print('obstacles_config:')
pprint(obstacles_config)

print('other_config:')
pprint(other_config)

Config path: ./config/test.yaml
Testing with the following parameters:
{'accumulate_data': True,
 'agent_config_path': './config/agent_test.yaml',
 'base_time': 1765457030,
 'build_path': './unity_build/1xlimotest_wind/UASRL.exe',
 'config_path': './config/test.yaml',
 'cuda': 0,
 'decision_frame_period': 5,
 'headless': False,
 'metrics_log_interval': 50,
 'metrics_smoothing': 0.9,
 'policy_name': 'multy_test_4050747',
 'save_path': './results/',
 'seed': 1234,
 'send_debug_action': True,
 'test_name': 'test_name',
 'torch_deterministic': True,
 'total_episodes': 11,
 'train_config_path': './config/train_limo.yaml',
 'worker_id': 0}
train_config:
{'actor_network_layers': [128, 128, 128],
 'agent_config_path': './config/agent.yaml',
 'alpha': 0.2,
 'alpha_lr': 0.0001,
 'autotune': True,
 'base_time': 1765457030,
 'batch_size': 256,
 'bootstrap': True,
 'bootstrap_batch_proportion': 0.8,
 'buffer_size': 100000,
 'build_path': './unity_build/3xlimo_wind/UASRL.exe',
 'cuda': 0,
 'env_id':

In [7]:
if torch.cuda.is_available() and args.cuda >= 0:
    # F-string per inserire l'indice: diventa "cuda:2"
    device_str = f"cuda:{args.cuda}"
else:
    device_str = "cpu"

DEVICE = torch.device(device_str)
print(f"Using device: {DEVICE}")

Using device: cuda:0


In [8]:
# seeding
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.backends.cudnn.deterministic = args.torch_deterministic
print(f'Seed: {args.seed}')

Seed: 1234


In [ ]:
BEHAVIOUR_NAME = other_config['behavior_name'] + '?team=' + other_config['team']

RAY_PER_DIRECTION = other_config['rays_per_direction']
RAYCAST_MIN = other_config['rays_min_observation']
RAYCAST_MAX = other_config['rays_max_observation']
RAYCAST_SIZE = 2*RAY_PER_DIRECTION + 1

STATE_SIZE = other_config['state_observation_size'] - 1
STATE_MIN = other_config['state_min_observation']
STATE_MAX = other_config['state_max_observation']

ACTION_SIZE = other_config['action_size']
ACTION_MIN = other_config['min_action']
ACTION_MAX = other_config['max_action']

TOTAL_STATE_SIZE = (STATE_SIZE + RAYCAST_SIZE)*train_config['input_stack']

In [ ]:
for p_name in args.policy_names:
    
    # Create the channel
    env_info = CustomChannel()
    param_channel = EnvironmentParametersChannel()

    for i in range(args.total_episodes):
        env_info.send_episode_seed(i+args.seed) # semplice seeding per ogni episodio

    # env setup
    print(f'Starting Unity Environment from build: {args.build_path}')
    # args.build_path
    env = UnityEnvironment(None, #args.build_path,
                        seed=args.seed,
                        side_channels=[env_info, param_channel], 
                        no_graphics=args.headless,
                        worker_id=args.worker_id)
    print('Unity Environment connected.')

    additional_number = int(time.time()) - train_config["base_time"]
    test_name = f"{args.test_name}_{additional_number}"
    args.test_full_name = test_name

    print(f"Test name: {args.test_full_name}")
    print(f"Policy name: {p_name}")
    
    # start training
    summary_save_filepath = args.save_path + args.test_name + ".csv"
    specific_save_filepath = args.save_path + args.test_name + "/" + f"{additional_number}" + ".json"
    os.makedirs(args.save_path, exist_ok=True)
    os.makedirs(args.save_path + args.test_name, exist_ok=True)
    
    print('Creating and loading actor and critic networks...')

    # ===== Actor =====
    actor = OldDenseActor(
        TOTAL_STATE_SIZE,
        ACTION_SIZE,
        ACTION_MIN,
        ACTION_MAX,
        train_config["actor_network_layers"]
    ).to(DEVICE)

    # ===== Q Ensemble =====
    '''qf_ensemble = [
        OldDenseSoftQNetwork(
            TOTAL_STATE_SIZE,
            ACTION_SIZE,
            train_config["q_network_layers"]
        ).to(DEVICE)
        for _ in range(train_config["q_ensemble_n"])
    ]

    qf_ensemble_target = [
        OldDenseSoftQNetwork(
            TOTAL_STATE_SIZE,
            ACTION_SIZE,
            train_config["q_network_layers"]
        ).to(DEVICE)
        for _ in range(train_config["q_ensemble_n"])
    ]'''

    # ===== Load saved weights =====
    load_models(actor, save_path='./models/' + p_name, suffix='_best')

    episodic_stats, dataset = test(env, 
            env_info,
            param_channel,
            
            args,
            agent_config,
            obstacles_config,
            
            actor, 
            
            BEHAVIOUR_NAME,
            STATE_SIZE,
            RAYCAST_SIZE,
            DEVICE)

    save_stats_to_csv({}, episodic_stats, summary_save_filepath)
    
    # Save dataset to JSON if accumulation is enabled
    if args.accumulate_data: 
        
        dataset = {
            'metadata': {
                'test_config': vars(args),
                'train_config': train_config,
                'agent_config': agent_config,
                'obstacles_config': obstacles_config,
                'other_config': other_config
            },
            'data': dataset
        }
        
        # Recursive helper to convert all numbers into float (JSON safe)
        def convert_all_to_float(obj):
            if isinstance(obj, dict):
                return {k: convert_all_to_float(v) for k, v in obj.items()}
            elif isinstance(obj, (list, tuple)):
                return [convert_all_to_float(item) for item in obj]
            elif isinstance(obj, (np.floating, Decimal)):
                return float(obj)
            else:
                return obj
            
        # Save dataset with timestamp in filename
        with open(specific_save_filepath, 'w+') as file:
            file.write(json.dumps(convert_all_to_float(dataset)))


# Close Environment

In [15]:
# close the environment
env.close()